In [1]:
import mdptoolbox
import numpy as np
from math import ceil, log

In [42]:
probToState1 = 0.89
valueEstimates = [0.0,0.0,8.4,14.3,7.3,4.2,0.0]
rews = [5.4,-1.2,0.0,-1.3,-2.7,-2.1,8.4]

In [43]:
number_of_states = len(valueEstimates)
probs = np.zeros((1, number_of_states, number_of_states))

probs[0, 0, 1] = probToState1
probs[0, 0, 2] = 1 - probToState1
for i in range(2, number_of_states-1):
    probs[0, i, i+1] = 1
probs[0, 1, 3] = 1
probs[0, -1, -1] = 1
probs

array([[[0.  , 0.89, 0.11, 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]]])

In [44]:
# rewards is action and state and also next state based 

rewards = np.zeros((1, number_of_states, number_of_states))

rewards[0,0,1] = rews[0]
rewards[0,0,2] = rews[1]
rewards[0,1,3] = rews[2]
rewards[0,2,3] = rews[3]
for i in range(3, number_of_states-1):
    rewards[0, i, i+1] = rews[i+1]
rewards

array([[[ 0. ,  5.4, -1.2,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , -1.3,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. , -2.7,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. , -2.1,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  8.4],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]]])

In [45]:
vi = mdptoolbox.mdp.ValueIteration(probs, rewards, 1)
vi.run()

In [46]:
vi.V

(8.131,
 3.6000000000000005,
 2.3000000000000007,
 3.6000000000000005,
 6.300000000000001,
 8.4,
 0.0)

In [47]:
### k-step estimate
# if k == 1 then only check the place we can go directly
k_list = []
for i in range(1, number_of_states):
    k_step = mdptoolbox.mdp.ValueIteration(probs, reward=rewards, discount=1, max_iter=i,\
                                   initial_value=valueEstimates)
    k_step.run()
    k_list.append(k_step.V[0])
k_list

[5.598000000000001, 18.831, 9.131, 3.931, 8.131, 8.131]

In [48]:
def td_lambda(k_list, lam):
    n_state = len(k_list)
    ret = 0
    for i in range(10000):
        if i < n_state:
            comp = (1-lam) * (lam**i) * k_list[i]
        else:
            comp = (1-lam) * (lam**i) * k_list[-1]
        ret += comp
    return ret

In [49]:
minn = 9999999
best = 0
trial = 100000
check = vi.V[0]
tdd = 0
for i in range(trial):
    lam = float(i) / trial
    can = td_lambda(k_list, lam)
    diff = abs(check - can)
#     if can > maks:
#         maks = can
#         best = lam
    
    minn, best, tdd = min((minn, best, tdd), (diff, lam, can))
print "result of lambda: ", tdd        
print "difference: ", minn
print "lambda: ", best

result of lambda:  8.13101398181
difference:  1.39818053029e-05
lambda:  0.2367


In [25]:
print td_lambda(k_list, 0.49567)

11.5939913522


In [13]:
maks = 0
best = 0
trial = 100000
check = vi.V[0]
tdd = 0
for i in range(trial):
    lam = float(i) / trial
    can = td_lambda(k_list, lam)
#     if can > maks:
#         maks = can
#         best = lam
    
    maks, best = max((maks, best), (can, lam))
print "max: ", maks
print "lambda: ", best

max:  5.5
lambda:  0.0
